# Identificação das OOVs

## Importações

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install editdistance

In [3]:
import ast
import numpy as np
import pandas as pd

import json
import codecs 
import editdistance

import gensim
from gensim.models import KeyedVectors

from tqdm import tqdm

## Leitura dos dados

In [4]:
df = pd.read_csv('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/fakenews_data_tokens.csv')

In [5]:
df['text_tokens'] = df['text_tokens'].apply(lambda x: ast.literal_eval(x))

len(df)

1383

In [6]:
df

,text,classification,text_tokens
0,China se beneficiou financeiramente do novo co...,0,"[china, beneficiou, financeiramente, coronavírus]"
1,Alimentos alcalinos ajudam a prevenir ou trata...,0,"[alimentos, alcalinos, ajudam, prevenir, trata..."
2,"Minas Gerais tem menos mortes que São Paulo, e...",0,"[minas, gerais, mortes, paulo, adotado, medida..."
3,"Quando os sintomas do COVID-19 aparecerem, as ...",0,"[sintomas, covid, aparecerem, tomar, azitromic..."
4,Beber água e fazer gargarejos com sal ou vinag...,0,"[beber, água, gargarejos, sal, vinagre, impede..."
...,...,...,...
1378,Vídeo é usado fora de contexto para relacionar...,1,"[vídeo, usado, contexto, relacionar, ação, pm,..."
1379,Vídeo engana ao atribuir protesto na Bélgica a...,1,"[vídeo, engana, atribuir, protesto, bélgica, p..."
1380,Vídeo mostra FBI confiscando máscaras de acumu...,1,"[vídeo, mostra, fbi, confiscando, máscaras, ac..."
1381,Vídeo usa informações falsas para dizer que lo...,1,"[vídeo, informações, falsas, lockdown, inútil,..."


## Leitura da Embedding

In [7]:
# EMBEDDING = KeyedVectors.load_word2vec_format('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/02. Word-Embeddings Pré-Treinadas/word2vec_cbow_s300.txt')
# EMBEDDING = KeyedVectors.load_word2vec_format('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/02. Word-Embeddings Pré-Treinadas/cbow_s50.txt')
EMBEDDING = KeyedVectors.load_word2vec_format('/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/02. Word-Embeddings Pré-Treinadas/cbow_s100.txt')
dim_embedding = 100

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [8]:
EMBEDDING

In [9]:
print("Tamanhos dos vocabulário da embedding: " + str(len(EMBEDDING.vocab)))

Tamanhos dos vocabulário da embedding: 929606


## Gerando vocabulário do dataset

In [10]:
def text_tokens_set(df):
  text_tokens = []
  text_tokens_set = []
  
  for tokens in df:
    text_tokens.extend(tokens)
      
  text_tokens_set.append((list(set(text_tokens))))
  return text_tokens_set

In [11]:
tokens_set = text_tokens_set(df['text_tokens'])[0]
print("Tamanhos do vocabulário do dataset: ", len(tokens_set))

Tamanhos do vocabulário do dataset:  3148


In [12]:
df

,text,classification,text_tokens
0,China se beneficiou financeiramente do novo co...,0,"[china, beneficiou, financeiramente, coronavírus]"
1,Alimentos alcalinos ajudam a prevenir ou trata...,0,"[alimentos, alcalinos, ajudam, prevenir, trata..."
2,"Minas Gerais tem menos mortes que São Paulo, e...",0,"[minas, gerais, mortes, paulo, adotado, medida..."
3,"Quando os sintomas do COVID-19 aparecerem, as ...",0,"[sintomas, covid, aparecerem, tomar, azitromic..."
4,Beber água e fazer gargarejos com sal ou vinag...,0,"[beber, água, gargarejos, sal, vinagre, impede..."
...,...,...,...
1378,Vídeo é usado fora de contexto para relacionar...,1,"[vídeo, usado, contexto, relacionar, ação, pm,..."
1379,Vídeo engana ao atribuir protesto na Bélgica a...,1,"[vídeo, engana, atribuir, protesto, bélgica, p..."
1380,Vídeo mostra FBI confiscando máscaras de acumu...,1,"[vídeo, mostra, fbi, confiscando, máscaras, ac..."
1381,Vídeo usa informações falsas para dizer que lo...,1,"[vídeo, informações, falsas, lockdown, inútil,..."


## Obtendo os oovs

In [13]:
def get_oovs(vocab, embedding):
  oovs = []
  
  for token in vocab:
    if token not in embedding.vocab:
      oovs.append(token)

  return oovs

In [14]:
OOVS = get_oovs(tokens_set, EMBEDDING)

In [15]:
print("Tamanhos dos OOVs: ", len(OOVS))

Tamanhos dos OOVs:  127


In [16]:
OOVS

['liberó',
 'distorsionan',
 'pisoetada',
 'evacuación',
 'ofreciendo',
 'agrediendo',
 'disminuyeron',
 'incautación',
 'coronavac',
 'asintomáticos',
 'adquisición',
 'estafadores',
 'inscriptas',
 'ofreció',
 'bloqueó',
 'cargado',
 'disfrazados',
 'kathlenn',
 'reciben',
 'enrojecidas',
 'alcanzó',
 'jambú',
 'aumentó',
 'tedros',
 'radarbox',
 'certillamado',
 'espancadoem',
 'mediocridad',
 'mortalidad',
 'ineficacia',
 'abordaje',
 'bolsonarista',
 'probada',
 'festejó',
 'transmiten',
 'firmó',
 'dándole',
 'autodiagnosticar',
 'descubrió',
 'multinacionales',
 'llenos',
 'comprueban',
 'groseras',
 'cargando',
 'fosas',
 'testeada',
 'posteó',
 'subnotificam',
 'afirmó',
 'vacíos',
 'muestre',
 'reportaje',
 'disminuyen',
 'monitorear',
 'cracolância',
 'ruilan',
 'confiscó',
 'distorsiona',
 'montaje',
 'prohibió',
 'conduzca',
 'propuso',
 'presidiarios',
 'cuarentenas',
 'sujetando',
 'cerró',
 'calló',
 'bolsonaristas',
 'sobornos',
 'informó',
 'suicidó',
 'donaciones',
 

## Tratamento dos oovs

In [17]:
oovs_path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/'

def namestr(obj, namespace):
  list_name = [name for name in namespace if namespace[name] is obj]
  return ' '.join([str(elem) for elem in list_name])


def treatment_oovs(oovs, word_embedding):
  dict_oovs = {}
  vocab_name = str(namestr(word_embedding, globals()))

  with tqdm(total=len(oovs), desc=vocab_name) as pbar:
    for word_oov in oovs:
      similar_words = []
      for word_vocab in list(word_embedding.vocab.keys()):
        similarity = editdistance.eval(word_oov, word_vocab)
        similar_words.append((word_vocab, similarity))
      dict_oovs[word_oov] = sorted(similar_words, key=lambda tup : tup[1])[:1][0][0]
      pbar.update(1)

  return dict_oovs

In [18]:
OOVS_EDIT = treatment_oovs(OOVS, EMBEDDING)
json.dump(OOVS_EDIT, codecs.open(oovs_path + 'oovs_100dim.json', 'w', encoding='utf-8'), separators=(',', ':'), sort_keys=True, indent=4)

___ EMBEDDING _8: 100%|██████████| 127/127 [03:36<00:00,  1.70s/it]


In [19]:
OOVS_EDIT

{'abordaje': 'abordade',
 'adhanom': 'phanom',
 'adoptó': 'adopta',
 'adquisición': 'inquisición',
 'afirmó': 'afirma',
 'aglomeración': 'aglomeracja',
 'agrediendo': 'agredindo',
 'aislamiento': 'isolamento',
 'alcanzó': 'alcanza',
 'almorzando': 'almoçando',
 'anciana': 'anciano',
 'aprobó': 'aprova',
 'arpen': 'aspen',
 'asaltantes': 'assaltantes',
 'asintomáticos': 'assintomáticos',
 'atiendan': 'atenda',
 'aumentó': 'aumento',
 'autodiagnosticar': 'autodiagnóstico',
 'barbijo': 'barbio',
 'bloqueó': 'bloqueá',
 'bolsonarista': 'bolsonaro',
 'bolsonaristas': 'bolsistas',
 'calló': 'call',
 'cargado': 'largado',
 'cargando': 'largando',
 'cavaron': 'camaron',
 'cerró': 'cerro',
 'certillamado': 'certificado',
 'combustible': 'combustibles',
 'comprueban': 'comprebem',
 'concluyó': 'concluiu',
 'conduzca': 'conduzia',
 'confinamiento': 'confinamento',
 'confiscó': 'confisco',
 'consiguiera': 'conseguira',
 'contagió': 'contagia',
 'coronavac': 'coroava',
 'covid': 'covil',
 'cracolân

# Tratamento das oovs e geração dos dados de treino e teste

In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[["text", "text_tokens"]], df["classification"], test_size=0.2, random_state=42)
len(X_train), len(X_test), len(y_train), len(y_test)

(1106, 277, 1106, 277)

In [28]:
X_train.index

Int64Index([ 886, 1206,  461,  374,  875, 1090, 1108,  432,  711,  744,
            ...
             330, 1238,  466,  121, 1044, 1095, 1130, 1294,  860, 1126],
           dtype='int64', length=1106)

In [29]:
y_train.index

Int64Index([ 886, 1206,  461,  374,  875, 1090, 1108,  432,  711,  744,
            ...
             330, 1238,  466,  121, 1044, 1095, 1130, 1294,  860, 1126],
           dtype='int64', length=1106)

In [21]:
def create_x(df):
    matrix = []

    for i, row in df.iterrows():
        resulting_vector = np.zeros(dim_embedding)
        tokens = row['text_tokens']
        for token in tokens:
            try:
                resulting_vector += EMBEDDING.get_vector(token)
            except:
                resulting_vector += EMBEDDING.get_vector(OOVS_EDIT[token])
        matrix.append(resulting_vector)
    return matrix


In [22]:
x_train_with_embedding = pd.DataFrame(create_x(X_train))
x_test_with_embedding = pd.DataFrame(create_x(X_test))
x_train_with_embedding['index'] = X_train.index
x_train_with_embedding['index'] = X_train.index
print(x_train_with_embedding.shape)
print(x_test_with_embedding.shape)

(1106, 100)
(277, 100)


In [24]:
x_train_with_embedding

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,1.380696,1.117377,0.137957,0.761084,0.534715,0.552442,-0.004785,0.152474,-1.161453,-0.055611,0.132354,-0.807359,-0.048969,0.822576,-0.392735,0.761977,-1.249718,-0.981962,-1.144687,0.978987,-0.320123,-0.708886,1.545143,0.277308,-0.886935,-0.091715,0.015212,1.188224,-0.579227,0.109587,0.628864,-0.653652,-1.361550,0.359014,0.358086,0.527734,0.379979,-0.916307,0.628407,0.948281,...,-0.139626,-1.028920,-0.002837,-1.452423,-0.445462,0.631283,-0.455308,-0.178009,-0.008535,-0.305238,0.184161,0.504119,0.575579,-0.408739,0.025195,-0.258042,-0.553967,-0.371115,-1.129306,-1.563835,-1.393074,0.807052,0.491048,0.359533,0.769408,-0.611857,-0.204234,0.872937,-0.285450,0.107422,0.427236,-0.796847,0.424916,0.063506,0.334243,-0.162930,0.768123,0.044175,-0.354515,0.294467
1,1.256561,-0.074913,0.145955,1.136830,0.004156,-0.870302,-1.605373,1.210091,0.282945,0.002317,-0.053097,0.487903,-0.638106,1.438257,0.339600,-0.661097,-0.211923,-1.258146,-0.357708,1.697232,-0.369264,0.859936,-0.557076,1.936877,-1.336052,-0.530775,0.261859,-0.710866,-0.785438,1.886991,-0.154471,-0.065372,-1.835043,-0.575359,-1.162260,0.632790,0.197085,-1.412120,-1.672758,0.127306,...,-1.585380,-0.303114,1.383609,-2.145113,0.153745,0.732192,-0.416609,0.694549,-1.263390,-1.328003,-0.351537,0.556735,0.410066,-0.833729,-1.081671,0.547176,0.297076,1.510936,-0.639551,0.223967,-1.502099,0.028978,1.489418,-0.645874,1.020926,-0.072282,-0.765184,1.740980,0.399647,1.177060,1.146743,-0.143019,-0.592275,0.039823,0.014546,0.103534,0.103899,0.478186,1.674268,1.021356
2,1.730329,0.072701,-0.001182,0.384270,0.467653,-0.644043,-0.718277,0.759209,1.150164,0.657574,-0.747905,0.727604,-0.094454,-0.702027,-0.745317,-0.037272,-1.346404,-0.873271,-2.331137,0.419611,-0.827884,1.455137,0.439708,2.281665,-1.272789,0.117314,1.122796,0.140376,0.022633,1.656343,0.727345,-0.550547,0.857728,-3.204981,-1.283571,2.088672,1.807675,-0.619398,-0.214990,0.144727,...,-0.642876,-1.383184,0.484584,-1.547997,0.054054,0.037525,0.964339,1.605726,-0.496296,-0.093266,0.642300,1.501297,0.197259,-1.480613,-0.577922,-0.850485,-0.065439,-1.388217,-2.119327,-1.221099,-0.591608,0.925732,0.328761,-0.430016,0.720029,-1.482438,-0.850712,1.561310,0.397910,0.820241,0.095608,0.235877,-0.841214,-0.966722,-0.783702,0.168322,0.311457,-0.297746,1.023306,-0.754920
3,-0.001383,-0.068355,-0.176925,0.785359,0.598387,0.482275,0.116769,1.513961,-0.517657,0.281069,-0.977379,1.359786,-0.103350,0.244982,-0.066079,-1.395389,-0.845733,-0.091960,0.151931,1.528891,0.338386,0.797338,0.254699,1.203377,-0.300761,-0.227034,0.784468,0.145645,0.430758,0.689033,1.378608,0.754982,0.689577,-0.696048,-1.606491,0.807161,0.621024,-2.588973,0.345826,0.778142,...,-0.893017,-1.001482,-0.333357,-0.619940,-0.722008,-0.088832,-0.160842,0.865481,-0.286700,-0.892267,1.494818,-0.566854,0.374465,-0.466815,-1.239538,-1.934977,0.195195,-1.389119,-1.112421,-0.776210,-1.135332,0.501908,1.364557,-0.345560,-0.202960,0.310760,-0.801793,0.208756,-0.053151,-0.710049,0.462919,-0.398455,-0.039614,0.865347,-0.418910,0.023014,0.069128,-0.570670,0.556745,0.301391
4,0.708433,-1.674911,-0.414836,0.604649,2.959635,0.938199,-0.836568,-0.111802,-0.676853,-0.356687,-0.307003,0.156009,-0.103471,1.101890,0.016957,-0.407898,-1.596707,0.408511,-0.668082,1.289135,-0.618351,0.795663,-0.317458,2.704531,-1.645182,-0.139670,0.375674,-0.407643,-0.265826,1.797180,1.028588,0.391431,0.765697,-2.112264,-0.527269,1.146115,1.394309,-1.534569,0.219000,0.609186,...,-0.740411,-0.015810,1.007980,-1.554050,-0.253225,-0.373098,1.658329,-0.466901,0.022074,0.167850,-0.929536,2.192057,-0.476363,-2.080670,-1.394679,-1.471324,-0.659242,-1.017550,-2.090855,-0.536676,-2.221824,-1.095451,1.896417,-0.168267,-0.835030,-1.185452,-1.194960,1.090425,0.132347,0.394848,0.448447,-1.555806,-1.08752

In [ ]:
path = '/content/drive/My Drive/[2020.1] APRENDIZADO DE MÁQUINA/TRABALHO/02. Arquivos/01. Dados/03. Dados para treino e teste 100dim/'
x_train_with_embedding.to_csv(path + 'x_train.csv')
x_test_with_embedding.to_csv(path + 'x_test.csv')
y_train.to_csv(path + 'y_train.csv')
y_test.to_csv(path + 'y_test.csv')